# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 9:48AM,238813,15,30009123,"Alliance Pharma, Inc.",Released
1,Jun 6 2022 9:48AM,238813,15,30009121,"Alliance Pharma, Inc.",Released
2,Jun 6 2022 9:48AM,238813,15,30009124,"Alliance Pharma, Inc.",Released
3,Jun 6 2022 9:48AM,238813,15,30009122,"Alliance Pharma, Inc.",Released
4,Jun 6 2022 9:48AM,238813,15,30009125,"Alliance Pharma, Inc.",Released
5,Jun 6 2022 9:48AM,238813,15,30009130,"Alliance Pharma, Inc.",Released
6,Jun 6 2022 9:48AM,238813,15,30009127,"Alliance Pharma, Inc.",Released
7,Jun 6 2022 9:48AM,238813,15,30009126,"Alliance Pharma, Inc.",Released
8,Jun 6 2022 9:48AM,238813,15,30009129,"Alliance Pharma, Inc.",Released
9,Jun 6 2022 9:48AM,238813,15,30009128,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,238843,Released,1
30,238844,Released,3
31,238845,Released,1
32,238846,Released,1
33,238847,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238843,NaN,NaN,1.0
238844,NaN,NaN,3.0
238845,NaN,NaN,1.0
238846,NaN,NaN,1.0
238847,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238813,0.0,0.0,28.0
238818,0.0,1.0,0.0
238819,2.0,21.0,6.0
238820,0.0,5.0,9.0
238821,0.0,2.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238813,0,0,28
238818,0,1,0
238819,2,21,6
238820,0,5,9
238821,0,2,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238813,0,0,28
1,238818,0,1,0
2,238819,2,21,6
3,238820,0,5,9
4,238821,0,2,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238813,,,28
1,238818,,1,
2,238819,2,21,6
3,238820,,5,9
4,238821,,2,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc."
28,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC"
31,Jun 6 2022 9:40AM,238846,10,Emerginnova
32,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation
93,Jun 6 2022 9:33AM,238845,12,Hush Hush
94,Jun 6 2022 9:28AM,238844,10,"Nextsource Biotechnology, LLC"
97,Jun 6 2022 9:25AM,238843,18,Eye Pharma Inc
98,Jun 6 2022 9:14AM,238839,10,ISDIN Corporation
106,Jun 6 2022 9:14AM,238841,12,Hush Hush
107,Jun 6 2022 9:13AM,238840,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc.",,,28
1,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC",,,3
2,Jun 6 2022 9:40AM,238846,10,Emerginnova,,,1
3,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation,,,61
4,Jun 6 2022 9:33AM,238845,12,Hush Hush,,,1
5,Jun 6 2022 9:28AM,238844,10,"Nextsource Biotechnology, LLC",,,3
6,Jun 6 2022 9:25AM,238843,18,Eye Pharma Inc,,,1
7,Jun 6 2022 9:14AM,238839,10,ISDIN Corporation,,,8
8,Jun 6 2022 9:14AM,238841,12,Hush Hush,,1,
9,Jun 6 2022 9:13AM,238840,12,Hush Hush,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc.",28,,
1,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC",3,,
2,Jun 6 2022 9:40AM,238846,10,Emerginnova,1,,
3,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation,61,,
4,Jun 6 2022 9:33AM,238845,12,Hush Hush,1,,
5,Jun 6 2022 9:28AM,238844,10,"Nextsource Biotechnology, LLC",3,,
6,Jun 6 2022 9:25AM,238843,18,Eye Pharma Inc,1,,
7,Jun 6 2022 9:14AM,238839,10,ISDIN Corporation,8,,
8,Jun 6 2022 9:14AM,238841,12,Hush Hush,,1,
9,Jun 6 2022 9:13AM,238840,12,Hush Hush,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc.",28,,
1,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC",3,,
2,Jun 6 2022 9:40AM,238846,10,Emerginnova,1,,
3,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation,61,,
4,Jun 6 2022 9:33AM,238845,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc.",28.0,NaN,NaN
1,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
2,Jun 6 2022 9:40AM,238846,10,Emerginnova,1.0,NaN,NaN
3,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation,61.0,NaN,NaN
4,Jun 6 2022 9:33AM,238845,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 9:48AM,238813,15,"Alliance Pharma, Inc.",28.0,0.0,0.0
1,Jun 6 2022 9:46AM,238847,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
2,Jun 6 2022 9:40AM,238846,10,Emerginnova,1.0,0.0,0.0
3,Jun 6 2022 9:37AM,238842,10,ISDIN Corporation,61.0,0.0,0.0
4,Jun 6 2022 9:33AM,238845,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3821335,411.0,5.0,0.0
12,955354,50.0,1.0,0.0
15,238813,28.0,0.0,0.0
18,238843,1.0,0.0,0.0
19,1194121,17.0,28.0,2.0
20,477673,11.0,0.0,0.0
22,238818,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3821335,411.0,5.0,0.0
1,12,955354,50.0,1.0,0.0
2,15,238813,28.0,0.0,0.0
3,18,238843,1.0,0.0,0.0
4,19,1194121,17.0,28.0,2.0
5,20,477673,11.0,0.0,0.0
6,22,238818,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,411.0,5.0,0.0
1,12,50.0,1.0,0.0
2,15,28.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,17.0,28.0,2.0
5,20,11.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,411.0
1,12,Released,50.0
2,15,Released,28.0
3,18,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,5.0,1.0,0.0,0.0,28.0,0.0,1.0
Released,411.0,50.0,28.0,1.0,17.0,11.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,5.0,1.0,0.0,0.0,28.0,0.0,1.0
2,Released,411.0,50.0,28.0,1.0,17.0,11.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,5.0,1.0,0.0,0.0,28.0,0.0,1.0
2,Released,411.0,50.0,28.0,1.0,17.0,11.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()